In [1]:
!date

Fri Mar  9 12:23:19 CST 2018


In [2]:
!ls ../source/gdppc_cppp/

README.md
gdppc_cppp-by-gapminder-v25-20180307.xlsx
gdppc_cppp-by-gapminder.xlsx
~$gdppc_cppp-by-gapminder.xlsx


In [3]:
!ls ../../

README.md
datapackage.json
ddf--concepts.csv
ddf--datapoints--gdp_per_capita_cppp--by--geo--time.csv
ddf--entities--geo.csv
etl


In [4]:
import pandas as pd

In [5]:
# about tab

In [50]:
about = pd.read_excel('../source/gdppc_cppp/gdppc_cppp-by-gapminder.xlsx', header=None)

In [51]:
about.head()

,0,1,2
0,About the data,NaN,code
1,Indicator name:,"GDP per capita, PPP (constant 2011 internation...",name
2,Indicator description:,GDP per capita based on purchasing power parit...,description
3,Source text short,"Gapminder based on World Bank, A. Maddison, M....",source_text_short
4,Source text long:,This data is compiled mainly from these source...,source_text_long


In [52]:
# code is not there yet. will add it manually

code = 'gdp_per_capita_cppp'

In [53]:
about = about.dropna(subset=[2], how='all')

In [54]:
about

,0,1,2
0,About the data,NaN,code
1,Indicator name:,"GDP per capita, PPP (constant 2011 internation...",name
2,Indicator description:,GDP per capita based on purchasing power parit...,description
3,Source text short,"Gapminder based on World Bank, A. Maddison, M....",source_text_short
4,Source text long:,This data is compiled mainly from these source...,source_text_long
5,Documentation link,gapm.io/dgdppc,source_link
6,Download latest version here:,https://github.com/Gapminder-Indicators/gdppc_...,link_download
9,Data version:,25,version
10,Updated on date:,"March 7, 2018",updated_date
11,Updated by:,Ola Rosling,updated_by


In [55]:
about = about.set_index(2)

In [56]:
measure = about[[1]].T

In [58]:
measure['code'] = code

In [59]:
measure = measure.set_index('code')

In [61]:
measure.index.name = 'concept'
measure.columns.name = None

measure

,name,description,source_text_short,source_text_long,source_link,link_download,version,updated_date,updated_by,about_file_generic,...,about_regions,about_gapminder_data,link_feedback,about_license,license_name,attribution_snippet,link_license,info_developers,permalink_file,permalink_repo
concept,,,,,,,,,,,,,,,,,,,,,
gdp_per_capita_cppp,"GDP per capita, PPP (constant 2011 internation...",GDP per capita based on purchasing power parit...,"Gapminder based on World Bank, A. Maddison, M....",This data is compiled mainly from these source...,gapm.io/dgdppc,https://github.com/Gapminder-Indicators/gdppc_...,25,"March 7, 2018",Ola Rosling,This file contains data compiled by Gapminder....,...,"To show the big picture, Gapminder has divided...",Here are Gapminder’s guidelines on reasonable ...,https://getsatisfaction.com/gapminder/products...,You can use this data in any way you like as l...,CC BY-4: Creative Common Attribution Licence 4.0,"""Free data from www.gapminder.org""",gapm.io/data_license,If you like to integrate lots of Gapminder dat...,https://github.com/Gapminder-Indicators/gdppc_...,https://github.com/Gapminder-Indicators/GDP-pe...


In [78]:
measure['concept_type'] = 'measure'

In [71]:
concepts = about[[0]].copy()

In [72]:
concepts

,0
concept,
code,About the data
name,Indicator name:
description,Indicator description:
source_text_short,Source text short
source_text_long,Source text long:
source_link,Documentation link
link_download,Download latest version here:
version,Data version:
updated_date,Updated on date:


In [73]:
concepts.columns = ['description']
concepts.index.name = 'concept'

In [74]:
concepts['description'] = concepts['description'].map(lambda x: x.replace(':', ''))

In [79]:
concepts['concept_type'] = 'string'

In [75]:
concepts

,description
concept,
code,About the data
name,Indicator name
description,Indicator description
source_text_short,Source text short
source_text_long,Source text long
source_link,Documentation link
link_download,Download latest version here
version,Data version
updated_date,Updated on date


In [154]:
domain = [ {'concept': 'geo', 'name': 'Geo', 'domain': None, 'concept_type': 'entity_domain'}
         , {'concept': 'country', 'name': 'Country', 'domain': 'geo', 'concept_type': 'entity_set'}
         , {'concept': 'world_4region', 'name': 'four regions', 'domain': 'geo', 'concept_type': 'entity_set'}
         , {'concept': 'global', 'name': 'global', 'domain': 'geo', 'concept_type': 'entity_set'}
         , {'concept': 'domain', 'name': 'Domain', 'concept_type': 'string'}
         ]

domain = pd.DataFrame.from_records(domain)

In [155]:
domain = domain.set_index('concept')

In [90]:
domain

,concept_type,domain,name
concept,,,
geo,entity_domain,None,Geo
country,entity_set,geo,Country
world_4region,entity_set,geo,four regions
global,entity_set,geo,global


In [91]:
time = pd.DataFrame.from_records([{'concept': 'year', 'concept_type': 'time', 'name': 'Year'}]).set_index('concept')

In [92]:
time

,concept_type,name
concept,,
year,time,Year


In [156]:
cdf = pd.concat([measure, concepts, domain, time])

In [157]:
cdf = cdf.reset_index()

In [158]:
cdf['name'] = cdf['name'].fillna(cdf['concept'].map(lambda x: x.replace('_', ' ').title()))

In [159]:
cdf

,concept,about_file_generic,about_gapminder_data,about_geo,about_license,about_regions,attribution_snippet,concept_type,description,domain,...,link_license,name,permalink_file,permalink_repo,source_link,source_text_long,source_text_short,updated_by,updated_date,version
0,gdp_per_capita_cppp,This file contains data compiled by Gapminder....,Here are Gapminder’s guidelines on reasonable ...,In order to make it easy for people to follow ...,You can use this data in any way you like as l...,"To show the big picture, Gapminder has divided...","""Free data from www.gapminder.org""",measure,GDP per capita based on purchasing power parit...,NaN,...,gapm.io/data_license,"GDP per capita, PPP (constant 2011 internation...",https://github.com/Gapminder-Indicators/gdppc_...,https://github.com/Gapminder-Indicators/GDP-pe...,gapm.io/dgdppc,This data is compiled mainly from these source...,"Gapminder based on World Bank, A. Maddison, M....",Ola Rosling,"March 7, 2018",25
1,code,NaN,NaN,NaN,NaN,NaN,NaN,string,About the data,NaN,...,NaN,Code,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,name,NaN,NaN,NaN,NaN,NaN,NaN,string,Indicator name,NaN,...,NaN,Name,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,description,NaN,NaN,NaN,NaN,NaN,NaN,string,Indicator description,NaN,...,NaN,Description,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,source_text_short,NaN,NaN,NaN,NaN,NaN,NaN,string,Source text short,NaN,...,NaN,Source Text Short,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,source_text_long,NaN,NaN,NaN,NaN,NaN,NaN,string,Source text long,NaN,...,NaN,Source Text Long,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,source_link,NaN,NaN,NaN,NaN,NaN,NaN,string,Documentation link,NaN,...,NaN,Source Link,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,link_download,NaN,NaN,NaN,NaN,NaN,NaN,string,Download latest version here,NaN,...,NaN,Link Download,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,version,NaN,NaN,NaN,NaN,NaN,NaN,string,Data version,NaN,...,NaN,Version,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,updated_date,NaN,NaN,NaN,NaN,NaN,NaN,string,Updated on date,NaN,...,NaN,Updated Date,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [160]:
cdf.to_csv('../../ddf--concepts.csv', index=False)

In [101]:
# datapoints

In [102]:
country_data = pd.read_excel('../source/gdppc_cppp/gdppc_cppp-by-gapminder.xlsx', sheet_name='countries_and_territories')

In [103]:
country_data.head()

,geo.name,indicator.name,geo,indicator,1800,1801,1802,1803,1804,1805,...,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040
0,Abkhazia,"GDP per capita, PPP (constant 2011 internation...",abkh,gdppc_cppp,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,"GDP per capita, PPP (constant 2011 internation...",afg,gdppc_cppp,603.0,603.0,603.0,603.0,603.0,603.0,...,2416.0,2469.0,2524.0,2580.0,2637.0,2695.0,2755.0,2816.0,2878.0,2942.0
2,Akrotiri and Dhekelia,"GDP per capita, PPP (constant 2011 internation...",akr_a_dhe,gdppc_cppp,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Albania,"GDP per capita, PPP (constant 2011 internation...",alb,gdppc_cppp,667.0,667.0,667.0,667.0,667.0,668.0,...,18471.0,18881.0,19299.0,19727.0,20164.0,20611.0,21068.0,21535.0,22012.0,22500.0
4,Algeria,"GDP per capita, PPP (constant 2011 internation...",dza,gdppc_cppp,715.0,716.0,717.0,718.0,719.0,720.0,...,15603.0,15948.0,16301.0,16662.0,17031.0,17408.0,17794.0,18188.0,18591.0,19003.0


In [105]:
# country entity
countries = country_data[['geo', 'geo.name']].copy()

In [107]:
countries.columns = ['country', 'name']

In [130]:
countries.to_csv('../../ddf--entities--geo--country.csv', index=False)

In [113]:
country_df = country_data.set_index('geo').drop(['geo.name', 'indicator', 'indicator.name'], axis=1)

In [114]:
country_df = country_df.stack().reset_index()

In [116]:
country_df.columns = ['country', 'year', code]

In [140]:
country_df.dropna(how='any').to_csv('../../ddf--datapoints--{}--by--country--year.csv'.format(code), index=False)

In [145]:
# 4 region
region_data = pd.read_excel('../source/gdppc_cppp/gdppc_cppp-by-gapminder.xlsx', 
                            sheet_name='four_regions')

In [146]:
region = region_data[['geo', 'geo.name']]
region.columns = ['world_4region', 'name']

In [147]:
region.to_csv('../../ddf--entities--geo--world_4region.csv', index=False)

In [152]:
region_df = region_data.set_index('geo').drop(['geo.name', 'indicator', 'indicator.name'], axis=1)
region_df = region_df.stack().reset_index()
region_df.columns = ['world_4region', 'year', code]

In [153]:
region_df.dropna(how='any').to_csv('../../ddf--datapoints--{}--by--world_4region--year.csv'.format(code), index=False)

In [127]:
# world

world_data = pd.read_excel('../source/gdppc_cppp/gdppc_cppp-by-gapminder.xlsx', 
                            sheet_name='world_total')

In [143]:
world = world_data[['geo', 'geo.name']]
world.columns = ['global', 'name']

world.to_csv('../../ddf--entities--geo--global.csv', index=False)

In [129]:
world_df = world_data.set_index('geo').drop(['geo.name', 'indicator', 'indicator.name'], axis=1)
world_df = world_df.stack().reset_index()
world_df.columns = ['global', 'year', code]

In [151]:
world_df.dropna(how='any').to_csv('../../ddf--datapoints--{}--by--global--year.csv'.format(code), index=False)

In [137]:
!ddf create_datapackage --overwrite ../../

backing up previous datapackage.json...
2018-03-12 14:02:31 WARNING skipping file .DS_Store
2018-03-12 14:02:31 WARNING skipping file datapackage.json
2018-03-12 14:02:31 WARNING skipping file .gitmodules
2018-03-12 14:02:31 WARNING skipping file README.md
2018-03-12 14:02:31 WARNING skipping file .gitignore
2018-03-12 14:02:31 WARNING skipping file datapackage.json.bak
2018-03-12 14:02:31 INFO generating ddf schema, may take some time...
2018-03-12 14:02:31 INFO loading dataset from disk: ddf--gapminder--gdp_per_capita_cppp
Traceback (most recent call last):
  File "pandas/_libs/parsers.pyx", line 1197, in pandas._libs.parsers.TextReader._convert_tokens
TypeError: Cannot cast array from dtype('O') to dtype('int64') according to the rule 'safe'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/semio/.pyenv/versions/3.6.2/bin/ddf", line 11, in <module>
    load_entry_point('ddf-utils', 'console_scripts', 'ddf')()
  Fi

In [136]:
!validate-ddf ../../

7root checking   0% /                                                           87dataset loading   0% /                                                         87dataset loading 100%[========================================================>]87dataset loading 100%[========================================================>]8/usr/local/lib/node_modules/ddf-validation/lib/shared.js:46
        const size = descriptorsGroup.reduce((sum, fileDescriptor) => sum + fileDescriptor.size, 0);
                                                                                           ^

TypeError: Cannot read property 'size' of undefined
    at descriptorsGroup.reduce (/usr/local/lib/node_modules/ddf-validation/lib/shared.js:46:92)
    at Array.reduce (<anonymous>)
    at exports.getDataPointFileDescriptorsGroups.map.descriptorsGroup (/usr/local/lib/node_modules/ddf-validation/lib/shared.js:46:39)
    at Array.map (<anonymous>)
    at Object.exports.getAllDataPointFileDescriptorsChunks (/usr/